## Матрицы в линейной регрессии

В нашем распоряжении данные о клиентах страховой компании. Пол, возраст, зарплата, члены семьи, страховые выплаты. Необходимо разработать такой метод преобразования данных, чтобы по ним было тяжело восстановить личные персональные данные.

Необходимо так преобразовать данные, чтобы качество метрики $R2$ было одинаковым для исходных и зашифрованных данных.

## 1. Загрузка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [2]:
data = pd.read_csv('insurance.csv')

In [3]:
data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [5]:
data.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


In [6]:
data['Страховые выплаты'].value_counts()

0    4436
1     423
2     115
3      18
4       7
5       1
Name: Страховые выплаты, dtype: int64

Пропусков нет. Все данные приведены к нужному типу.

## 2. Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Формула предсказаний:

$$
a = Xw
$$

Задача обучения - минимизация функции потерь:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения линейной регрессии:

$$
w = (X^T X)^{-1} X^T y
$$

Качество линейной регрессии не изменится при умножении признаков на обратимую матриицу.

Пусть $Р$ - обратимая матрица, a $a_{end}$ - предсказания после преобразования.  
Тогда будет верно равенство $a_{end} = X_{1}w_{1} = Xw = a$,  
где $X_{1} = XP$,  
а $w_{1}$ - вектор весов, в котором все признаки умножены на обратимую матрицу.


## 3. Алгоритм преобразования

**Алгоритм**

Будем обучать линейную регрессию на исходных признаках $X$ и на признаках, умноженных на обратимую матрицу $XP$. Затем сравним качество предсказания. Качество не должно измениться.

Ниже докажем, что умножение признаков на обратимую матрицу не изменит качества предсказаний линейной регрессии.


**Доказательство**

$$a_{end}=XP((XP)^T(XP))^{-1}(XP)^Ty=XP(P^TX^TXP)^{-1}P^TX^Ty=  
(P^T)^{-1}XP(X^TXP)^{-1}P^TX^Ty=XPP^{-1}(X^TX)^{-1}(P^T)^{-1}P^TX^Ty=XPP^{-1}(X^TX)^{-1}(P^T)^{-1}P^TX^Ty=
XE(X^TX)^{-1}EX^Ty=X(X^TX)^{-1}X^Ty=a$$

## 4. Проверка алгоритма

Выделим признаки $X$ и целевой признак $y$

In [7]:
X = data.drop('Страховые выплаты', axis=1)
y = data['Страховые выплаты']

Преобразуем признаки в вектор и матрицу

In [8]:
X = X.values
y = y.values

Обучим модель линейной регрессии на этих признаках

In [9]:
model = LinearRegression()
model.fit(X,y)
predictions = model.predict(X)

In [10]:
r2_original = r2_score(y, predictions)
print(r2_original)

0.4249455028666801


In [11]:
X.shape

(5000, 4)

Построим обратимую матрицу P

In [12]:
P = np.random.normal(size=(4, 4))

In [13]:
P.shape

(4, 4)

Проверим матрицу $P$ на обратимость. Для этого умножим ее на обратную себе $P^{-1}$. В результате должны получить единичную матрицу $E$.

In [14]:
E = P @ np.linalg.inv(P)

In [15]:
print(np.round(E))

[[ 1.  0.  0.  0.]
 [-0.  1.  0. -0.]
 [ 0. -0.  1. -0.]
 [-0.  0. -0.  1.]]


Результат умножения случайной обратимой матрицы на обратную действительно равен единичной матрице. Матрица $P$ - обратима.

Умножим матрицу признаков $X$ на случайную обратимую матрицу $P$

In [16]:
X_ = X @ P

Обучим модель линейной регрессии на измененных признаках

In [17]:
model.fit(X_,y)
predictions = model.predict(X_)

In [18]:
r2_changed = r2_score(y, predictions)
print(r2_changed)

0.42494550286669086


### Вывод

В задании доказано свойство неизменности предсказаний линейной регрессии при умножении вектора признаков на любую обратимую матрицу.  
Были обучены две модели: на основании исходных признаков $X$ и по измененным признакам $X\_$.  
Значения метрик $R2$ полностью совпадают.